In [13]:
import requests
import uuid
from RandomFunctions import Random_FHIR_Person
from RandomFunctions import Random_FHIR_ServiceRequest
from requests.auth import HTTPBasicAuth
from fhir.resources.R4B.bundle import Bundle

#fhir_server_url = 'https://spark.incendi.no/fhir'
#fhir_server_url = 'https://server.fire.ly'
#fhir_server_url = 'http://fhir.oridashi.com.au'
#fhir_server_url = 'https://demo.kodjin.com/fhir'
#Works R5# fhir_server_url = 'http://fhir.training.hl7.org.au/fhir'
    #fhir_server_url = 'https://demo.pathling.app/fhir'
    #fhir_server_url = 'https://fhir.hl7.org.au/ereq/fhir/DEFAULT' #sparked
fhir_server_url = "https://pyroserver.azurewebsites.net/pyro" # sonic


headers = {'Content-Type': 'application/fhir+json'}

# Create a patient
PersonGenerator = Random_FHIR_Person()
ServiceGenerator = Random_FHIR_ServiceRequest()


ValueSet Expansion: 3437 members
ValueSet Expansion: 1278 members


In [47]:
import uuid
import requests
from fhir.resources.R4B.bundle import Bundle
from fhir.resources.R4B.narrative import Narrative
from fhir.resources.R4B.patient import Patient
from fhir.resources.R4B.practitioner import Practitioner
from fhir.resources.R4B.servicerequest import ServiceRequest
import time

# Function to create a narrative
def create_narrative(resource):
    return Narrative(
        status="generated",
        div=f"<div xmlns='http://www.w3.org/1999/xhtml'>{resource.__class__.__name__} resource</div>"
    )

# Loop through as many times as you want to generate and post randomised data
for _ in range(30):
    # Generate some randomised data
    patient = PersonGenerator.NewPatient()
    practitioner = PersonGenerator.NewPractitioner()
    referral = ServiceGenerator.NewRandomRequest(patient=patient, requester=practitioner)

    # Ensure the IDs are valid UUIDs
    patient.id = str(uuid.uuid4())
    practitioner.id = str(uuid.uuid4())
    referral.id = str(uuid.uuid4())

    # Add narrative to each resource
    patient.text = create_narrative(patient)
    practitioner.text = create_narrative(practitioner)
    referral.text = create_narrative(referral)

    # Ensure the references in the referral are correctly set
    referral.subject.reference = f'urn:uuid:{patient.id}'
    referral.requester.reference = f'urn:uuid:{practitioner.id}'

    # Post a bundle with ServiceRequest and referenced patient+requester
    bundle = Bundle(
        type='transaction',
        id=str(uuid.uuid4()),
        entry=[
            {
                'fullUrl': f'urn:uuid:{patient.id}',
                'resource': patient,
                'request': {'method': 'PUT', 'url': f'Patient/{patient.id}'}
            },
            {
                'fullUrl': f'urn:uuid:{practitioner.id}',
                'resource': practitioner,
                'request': {'method': 'PUT', 'url': f'Practitioner/{practitioner.id}'}
            },
            {
                'fullUrl': f'urn:uuid:{referral.id}',
                'resource': referral,
                'request': {'method': 'PUT', 'url': f'ServiceRequest/{referral.id}'}
            }
        ]
    )

    payload = bundle.json()
    print(payload)

    #Sparked Credentials
    username = "placer"
    password = "ps8qs7kLVbjS5Gr"


    response = requests.post(fhir_server_url, data=payload, headers={'Content-Type': 'application/json'}, auth=HTTPBasicAuth(username, password))
    #print(response.text)

    # Pause for 5 seconds
    time.sleep(2)

print('Done')

{"resourceType": "Bundle", "id": "0228b74a-fa8c-47ea-97f4-43300afdb3fb", "type": "transaction", "entry": [{"fullUrl": "urn:uuid:b617cbd9-0b91-4f33-8f04-b5ad81f78086", "resource": {"resourceType": "Patient", "id": "b617cbd9-0b91-4f33-8f04-b5ad81f78086", "text": {"status": "generated", "div": "<div xmlns='http://www.w3.org/1999/xhtml'>Patient resource</div>"}, "name": [{"text": "Ambitious Gouldian Finch", "family": "Gouldian Finch", "given": ["Ambitious"]}], "gender": "female", "birthDate": "1986-07-27", "address": [{"text": "101 Leptospermum Hwy, Brunswick, TAS 1908", "line": ["101 Leptospermum Hwy"], "city": "Brunswick", "state": "TAS", "postalCode": "1908", "country": "Australia"}]}, "request": {"method": "PUT", "url": "Patient/b617cbd9-0b91-4f33-8f04-b5ad81f78086"}}, {"fullUrl": "urn:uuid:c5fafcba-3d36-4b04-99d9-dc9392cd5f5d", "resource": {"resourceType": "Practitioner", "id": "c5fafcba-3d36-4b04-99d9-dc9392cd5f5d", "text": {"status": "generated", "div": "<div xmlns='http://www.w3.or

In [31]:
from flask import Flask, render_template_string
import requests
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta


# Calculate the time 6 hours ago
six_hours_ago = datetime.utcnow() - timedelta(hours=6)
six_hours_ago_str = six_hours_ago.strftime('%Y-%m-%dT%H:%M:%SZ')

# Endpoint to retrieve all ServiceRequest resources created in the last 6 hours, sorted by most recent
service_request_endpoint = f"{fhir_server_url}/ServiceRequest?_lastUpdated=gt{six_hours_ago_str}&_sort=-_lastUpdated&_count=15"

# Make the GET request with basic authentication
response = requests.get(service_request_endpoint, auth=HTTPBasicAuth(username, password))

C:\Users\cor247\AppData\Local\Temp\ipykernel_13500\3787969020.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  six_hours_ago = datetime.utcnow() - timedelta(hours=6)


In [36]:
service_requests = response.json().get('entry', [])
service_requests = [entry['resource'] for entry in service_requests]

service_requests[0]

{'resourceType': 'ServiceRequest',
 'id': 'f5bc5777-f97b-4bee-b108-59182e2b622c',
 'meta': {'versionId': '1',
  'lastUpdated': '2024-12-13T02:46:42.9129659+00:00'},
 'text': {'status': 'generated',
  'div': '<div xmlns="http://www.w3.org/1999/xhtml">ServiceRequest resource</div>'},
 'status': 'active',
 'intent': 'order',
 'category': [{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/service-category',
     'code': '108252007',
     'display': 'Laboratory procedure'}]}],
 'priority': 'routine',
 'code': {'coding': [{'system': 'http://snomed.info/sct',
    'code': '117617002',
    'display': 'Immunohistochemistry procedure'}]},
 'subject': {'reference': 'https://pyroserver.azurewebsites.net/pyro/Patient/e36a8115-362a-4baf-95c8-5501444bea94'},
 'requester': {'reference': 'https://pyroserver.azurewebsites.net/pyro/Practitioner/71b44206-580d-4d89-93fd-80959bb65761'}}

In [43]:
for service_request in service_requests:
            subject_reference = service_request.get('subject', {}).get('reference')
            #print(subject_reference)
            if subject_reference:
                patient_response = requests.get(f"{subject_reference}", auth=HTTPBasicAuth(username, password))
                print(patient_response)

                if patient_response.status_code == 200:
                    patient = patient_response.json()
                    service_request['subject_name'] = patient.get('name', [{}])[0].get('text', 'N/A')
                    service_request['subject_gender'] = patient.get('gender', 'N/A')
                else:
                    service_request['subject_name'] = 'N/A'
                    service_request['subject_gender'] = 'N/A'
            else:
                service_request['subject_name'] = 'N/A'
                service_request['subject_gender'] = 'N/A'

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
